## Gerendo previsões com novos dados (simulando um cenário de produção)

In [1]:
# Carregando os pacotes que serão utilizados
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from category_encoders.one_hot import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, accuracy_score

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Carregando o novo arquivo com dados novos
df = pd.read_csv('novos_dados.csv')

In [5]:
# Renomeando as colunas
df.columns = ['ID_Funcionario', 
              'Idade', 
              'Genero', 
              'Anos_Empresa', 
              'Funcao', 
              'Salario_Mensal',
              'Equilibrio_Vida',
              'Satisfacao',
              'Desempenho',
              'Qtd_Promocoes',
              'Hora_Extra',
              'Distancia_Casa',
              'Escolaridade',
              'Estado_Civil',
              'Qtd_Dependentes',
              'Cargo',
              'Tamanho_Empresa',
              'Qtd_Anos_Trabalho',
              'Trabalho_Remoto',
              'Oportunidade_Lideranca',
              'Oportunidade_Inovacao',
              'Reputacao_Empresa',
              'Reconhecimento']              

In [7]:
# Fazendo uma cópia do DataFrame
novos_dados = df.copy()

In [9]:
# Fazendo o tratamento de dados igual fizemos para a criação e treinamento do modelo.
# Importante: Todo tratamento de dado que for realizado nos dados na etapa de construção do modelo devem ser feitos também em dados novos

novos_dados = novos_dados.drop(columns=['ID_Funcionario'])

valores = [-100, 5000, 10000, 20000]
rotulos = ['Até $5.000', 'De $5.001 até $10.000 ', 'Acima de $10.000']
novos_dados['Faixa_Salarial'] = pd.cut(novos_dados['Salario_Mensal'], bins=valores, labels=rotulos)

novos_dados = novos_dados.drop(columns=['Salario_Mensal'])

novos_dados = novos_dados.drop(columns=['Qtd_Anos_Trabalho'])
novos_dados.loc[(novos_dados['Idade'] - novos_dados['Anos_Empresa'] < 18), 'Anos_Empresa'] = 1


variaveis_categoricas_ordinais = ['Faixa_Salarial','Equilibrio_Vida','Satisfacao','Desempenho','Escolaridade','Cargo',
                                 'Tamanho_Empresa','Reputacao_Empresa','Reconhecimento']

encoder = OrdinalEncoder()
for var in variaveis_categoricas_ordinais:
    novos_dados[var] = encoder.fit_transform(novos_dados[var])

variaveis_categoricas_nominais = ['Genero','Funcao','Hora_Extra','Estado_Civil','Trabalho_Remoto','Oportunidade_Lideranca',
                                 'Oportunidade_Inovacao']


encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(novos_dados[variaveis_categoricas_nominais])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(variaveis_categoricas_nominais))
novos_dados = pd.concat([novos_dados, one_hot_df], axis=1)
novos_dados = novos_dados.drop(variaveis_categoricas_nominais, axis=1)

In [11]:
# Visualizando informações do nosso conjunto de dados após as alterações
novos_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14900 entries, 0 to 14899
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Idade                       14900 non-null  int64  
 1   Anos_Empresa                14900 non-null  int64  
 2   Equilibrio_Vida             14900 non-null  int32  
 3   Satisfacao                  14900 non-null  int32  
 4   Desempenho                  14900 non-null  int32  
 5   Qtd_Promocoes               14900 non-null  int64  
 6   Distancia_Casa              14900 non-null  int64  
 7   Escolaridade                14900 non-null  int32  
 8   Qtd_Dependentes             14900 non-null  int64  
 9   Cargo                       14900 non-null  int32  
 10  Tamanho_Empresa             14900 non-null  int32  
 11  Reputacao_Empresa           14900 non-null  int32  
 12  Reconhecimento              14900 non-null  int32  
 13  Faixa_Salarial              149

In [13]:
# Separando as variaveis preditoras que serão utilizadas no modelo

#PREDITORAS = novos_dados.iloc[:, 0:32]

PREDITORAS = novos_dados.copy()
PREDITORAS.head()

,Idade,Anos_Empresa,Equilibrio_Vida,Satisfacao,Desempenho,Qtd_Promocoes,Distancia_Casa,Escolaridade,Qtd_Dependentes,Cargo,Tamanho_Empresa,Reputacao_Empresa,Reconhecimento,Faixa_Salarial,Genero_Female,Genero_Male,Funcao_Education,Funcao_Finance,Funcao_Healthcare,Funcao_Media,Funcao_Technology,Hora_Extra_No,Hora_Extra_Yes,Estado_Civil_Divorced,Estado_Civil_Married,Estado_Civil_Single,Trabalho_Remoto_No,Trabalho_Remoto_Yes,Oportunidade_Lideranca_No,Oportunidade_Lideranca_Yes,Oportunidade_Inovacao_No,Oportunidade_Inovacao_Yes
0,36,13,1,1,1,1,83,1,1,1,1,1,1,2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,35,7,2,1,1,1,55,2,4,2,2,2,2,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2,50,7,3,1,1,3,14,2,2,3,2,2,3,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,58,1,3,2,2,0,43,1,4,2,2,1,3,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,39,1,2,1,1,0,47,1,6,1,1,2,2,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [15]:
Normalizador = MinMaxScaler()
dados_normalizados = Normalizador.fit_transform(PREDITORAS)    

clf = joblib.load('modelo_treinado_desligamento_funcionarios.pk')


previsoes = clf.predict(dados_normalizados)
probabilidades = clf.predict_proba(dados_normalizados)

df['PREVISOES'] = previsoes
df['PROBABILIDADES'] = probabilidades[:, 1]       

df.to_excel('previsoes_desligamentos.xlsx')
print("Previsoes Geradas com Sucesso!")        

Previsoes Geradas com Sucesso!
